# BERT Tokenizer

Der BERT Tokenizer ist ein Tokenizer, der mit BERT arbeitet. Er hat viele Funktionen für alle Arten von Tokenisierungsaufgaben. Sie können den Tokenizer mit dieser Codezeile herunterladen:

In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Im Gegensatz zu den BERT-Modellen müssen Sie nicht für jede Art von Modell einen anderen Tokenizer herunterladen. Sie können denselben Tokenizer für alle verschiedenen BERT-Modelle verwenden, die hugging face bietet.
Bei einer Texteingabe gehe ich im Allgemeinen so vor, dass ich sie in Projekten tokenisiere:

In [ ]:
encoding = tokenizer.encode_plus(text, add_special_tokens = True,
                                 truncation = True, padding = "max_length", 
                                 return_attention_mask = True, 
                                 return_tensors = "pt")

Da BERT jeweils nur 512 Token als Eingabe akzeptieren/aufnehmen kann, muss der Parameter truncation auf True gesetzt werden. Der Parameter add special tokens dient nur dazu, dass BERT Token wie die Start-, End-, [SEP]- und [CLS]-Token hinzufügen kann. Return_tensors = "pt" dient nur dazu, dass der Tokenizer PyTorch-Tensoren zurückgibt. Wenn Sie das nicht wollen (vielleicht wollen Sie, dass er eine Liste zurückgibt), können Sie den Parameter entfernen und er gibt Listen zurück.

Im folgenden Code werden Sie sehen, dass ich nicht alle oben aufgeführten Parameter hinzugefügt habe. Das liegt in erster Linie daran, dass dies nicht notwendig ist, da ich den Text nicht für ein echtes Projekt tokenisiere. In einem echten maschinellen Lern-/NLP-Projekt sollten Sie diese Parameter hinzufügen, insbesondere die Trunkierung und das Auffüllen, da wir dies für jeden Stapel im Datensatz in einem echten Projekt tun müssen.

tokenizer.encode_plus() gibt speziell ein Wörterbuch mit Werten zurück und nicht nur eine Liste von Werten. Da tokenizer.encode_plus() viele verschiedene Arten von Informationen zurückgeben kann, wie z. B. die attention_masks und Token-Typ-IDs, wird alles in einem Wörterbuchformat zurückgegeben, und wenn Sie die spezifischen Teile der Kodierung abrufen möchten, können Sie dies wie folgt tun:

In [ ]:
input = encoding["input_ids"][0]
attention_mask = encoding["attention_mask"][0]

Da der Tokenizer außerdem ein Wörterbuch mit verschiedenen Werten zurückgibt, können wir, anstatt diese Werte wie oben gezeigt zu suchen und einzeln in das Modell zu übertragen, einfach die gesamte Kodierung wie folgt übergeben

In [ ]:
output = model(**encoding)

Ein weiterer wichtiger Aspekt des Tokenizers ist, dass Sie angeben können, dass bestimmte Token abgerufen werden sollen, falls gewünscht. Wenn Sie beispielsweise eine maskierte Sprachmodellierung durchführen und eine Maske an einer Stelle einfügen möchten, die Ihr Modell dekodieren soll, dann können Sie das Maskentoken einfach wie folgt abrufen

In [ ]:
mask_token = tokenizer.mask_token

und Sie können es einfach in Ihre Eingabe einfügen, indem Sie es mit Ihrem Eingabetext verketten.
Auf die gleiche Weise können Sie auch viele andere Token abrufen, wie z. B. das [SEP]-Token.

Normalerweise verwende ich die Funktion tokenizer.encode_plus(), um meine Eingaben zu tokenisieren, aber es gibt noch eine weitere Funktion, die verwendet werden kann, um Eingaben zu tokenisieren, und zwar tokenizer.encode(). Hier ist ein Beispiel dafür:

In [ ]:
encoding = tokenizer.encode(text, return_tensors = "pt")

Der Hauptunterschied zwischen tokenizer.encode_plus() und tokenizer.encode() ist, dass tokenizer.encode_plus() mehr Informationen zurückgibt. Insbesondere werden die tatsächlichen Eingabe-Ids, die Aufmerksamkeitsmasken und die Token-Typ-Ids zurückgegeben, und zwar in einem Wörterbuch. tokenizer.encode() gibt nur die Eingabe-Ids zurück, und zwar entweder als Liste oder als Tensor, je nach dem Parameter return_tensors = "pt".


# Masked Language Modeling

Masked Language Modeling ist die Aufgabe, ein maskiertes Token in einem Satz zu dekodieren. Vereinfacht ausgedrückt ist es die Aufgabe, die Lücken zu füllen.

Anstatt einfach nur das beste Kandidatenwort für die Ersetzung des Masken-Tokens zu ermitteln, zeige ich Ihnen, wie Sie die 10 besten Ersetzungswörter für das Masken-Token ermitteln können, und zwar folgendermaßen:

In [6]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)
text = "The capital of France, " + tokenizer.mask_token + ", contains the Eiffel Tower."
input = tokenizer.encode_plus(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
output = model(**input)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The capital of France, paris, contains the Eiffel Tower.
The capital of France, lyon, contains the Eiffel Tower.
The capital of France, lille, contains the Eiffel Tower.
The capital of France, toulouse, contains the Eiffel Tower.
The capital of France, marseille, contains the Eiffel Tower.
The capital of France, orleans, contains the Eiffel Tower.
The capital of France, strasbourg, contains the Eiffel Tower.
The capital of France, nice, contains the Eiffel Tower.
The capital of France, cannes, contains the Eiffel Tower.
The capital of France, versailles, contains the Eiffel Tower.


Hugging Face ist so aufgebaut, dass Sie für die Aufgaben, für die es bereits trainierte Modelle hat, dieses spezifische Modell herunterladen/importieren müssen. In diesem Fall müssen wir das Modell "Bert For Masked Language Modeling" herunterladen, während der Tokenizer für alle verschiedenen Modelle gleich ist, wie ich bereits im obigen Abschnitt erwähnt habe.

Bei der maskierten Sprachmodellierung wird ein Maskentoken an der gewünschten Stelle eingefügt, an der Sie das beste Kandidatenwort vorhersagen möchten, das an dieser Stelle stehen würde. Sie können das Masken-Token einfach einfügen, indem Sie es an der gewünschten Position in Ihrer Eingabe verketten, wie ich es oben getan habe. Das Bert-Modell für maskierte Sprachmodellierung sagt das beste Wort/Token in seinem Vokabular voraus, das dieses Wort ersetzen würde. Die Logits sind die Ausgabe des BERT-Modells, bevor eine Softmax-Aktivierungsfunktion auf die Ausgabe von BERT angewendet wird. Um die Logits zu erhalten, müssen wir bei der Initialisierung des Modells in den Parametern return_dict = True angeben, sonst führt der obige Code zu einem Kompilierungsfehler. Nachdem wir die Eingabekodierung an das BERT-Modell übergeben haben, können wir die Logits einfach durch Angabe von output.logits erhalten, das einen Tensor zurückgibt, und danach können wir schließlich eine Softmax-Aktivierungsfunktion auf die Logits anwenden. Durch die Anwendung einer Softmax auf die Ausgabe von BERT erhalten wir probabilistische Verteilungen für jedes der Wörter im BERT-Wortschatz. Wörter mit einem höheren Wahrscheinlichkeitswert sind bessere Kandidaten für die Ersetzung des Maskentokens. Um den Tensor der Softmax-Werte aller Wörter im BERT-Vokabular für die Ersetzung des Maskentokens zu erhalten, können wir den Index des maskierten Tokens angeben, den wir mit torch.where() erhalten. Da ich in diesem Beispiel die 10 besten Kandidaten für die Ersetzung des maskierten Tokens abrufe (Sie können mehr als 10 erhalten, indem Sie den Parameter entsprechend anpassen), habe ich die Funktion torch.topk() verwendet, mit der Sie die obersten k Werte in einem gegebenen Tensor abrufen können, und sie gibt einen Tensor mit diesen obersten k Werten zurück. Danach wird der Prozess relativ einfach, da wir nur durch den Tensor iterieren und das Masken-Token im Satz durch das Kandidaten-Token ersetzen müssen.

Sie können sehen, dass Paris tatsächlich der Top-Kandidat für die Ersetzung des Maskentokens ist.
Wenn Sie nur das Top-Kandidatenwort erhalten möchten, können Sie dies tun:

In [8]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)
text = "The capital of France, " + tokenizer.mask_token + " ,contains the Eiffel Tower."
input = tokenizer.encode_plus(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
logits = model(**input)
logits = logits.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_word = torch.argmax(mask_word, dim=1)
print(tokenizer.decode(top_word))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


paris


Anstatt torch.topk() zum Abrufen der 10 höchsten Werte zu verwenden, benutzen wir einfach torch.argmax(), das den Index des höchsten Wertes im Tensor zurückgibt. Der Rest des Codes ist so ziemlich das Gleiche wie der Originalcode.

# Language Modeling

Bei der Sprachmodellierung geht es darum, das beste Wort für die Fortsetzung eines Satzes aus allen bereits im Satz enthaltenen Wörtern zu ermitteln.

In [10]:
import transformers
from transformers import BertTokenizer, BertLMHeadModel
import torch
from torch.nn import functional as F
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertLMHeadModel.from_pretrained('bert-base-uncased',
return_dict=True, is_decoder = True)
text = "A knife is very "
input = tokenizer.encode_plus(text, return_tensors = "pt")
output = model(**input).logits[:, -1, :]
softmax = F.softmax(output, -1)
index = torch.argmax(softmax, dim = -1)
x = tokenizer.decode(index)
print(x)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


.


Die Sprachmodellierung funktioniert ganz ähnlich wie die maskierte Sprachmodellierung. Zu Beginn müssen wir das spezielle Bert Language Model Head Model herunterladen, das im Wesentlichen ein BERT-Modell mit einem Sprachmodellierungskopf darauf ist. Ein zusätzlicher Parameter, den wir bei der Instanziierung dieses Modells angeben müssen, ist der Parameter is_decoder = True. Wir müssen diesen Parameter angeben, wenn wir dieses Modell als eigenständiges Modell für die Vorhersage des nächstbesten Wortes in der Sequenz verwenden wollen. Der Rest des Codes ist relativ identisch mit dem der maskierten Sprachmodellierung: Wir müssen die Logits des Modells abrufen, aber anstatt den Index des maskierten Tokens anzugeben, müssen wir nur die Logits des letzten versteckten Zustands des Modells nehmen (unter Verwendung des Index -1), die Softmax dieser Logits berechnen, den größten Wahrscheinlichkeitswert im Vokabular finden und dieses Token dekodieren und ausgeben.

# Next Sentence Prediction

Die Vorhersage des nächsten Satzes ist die Aufgabe, vorherzusagen, ob ein Satz auf einen anderen Satz folgt. Hier ist mein Code für diese Aufgabe:

In [11]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
from torch.nn import functional as F
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
prompt = "The child came home from school."
next_sentence = "He played soccer after school."
encoding = tokenizer.encode_plus(prompt, next_sentence, return_tensors='pt')
outputs = model(**encoding)[0]
softmax = F.softmax(outputs, dim = 1)
print(softmax)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[0.9953, 0.0047]], grad_fn=<SoftmaxBackward0>)


Bei der Vorhersage des nächsten Satzes geht es darum, vorherzusagen, wie gut ein Satz als nächster Satz für einen gegebenen Satz geeignet ist. In diesem Fall ist "Das Kind kam von der Schule nach Hause." der gegebene Satz und wir versuchen vorherzusagen, ob "Er spielte nach der Schule Fußball." der nächste Satz ist. Zu diesem Zweck fügt der BERT-Tokenizer automatisch ein [SEP]-Token zwischen den Sätzen ein, das die Trennung zwischen den beiden Sätzen darstellt, und das spezielle Bert-Modell für die Vorhersage des nächsten Satzes sagt zwei Werte dafür voraus, ob der Satz der nächste Satz ist. Bert gibt zwei Werte in einem Tensor zurück: Der erste Wert gibt an, ob der zweite Satz eine Fortsetzung des ersten ist, und der zweite Wert gibt an, ob der zweite Satz eine zufällige Folge oder keine gute Fortsetzung des ersten ist. Anders als bei der Sprachmodellierung rufen wir keine Logits ab, weil wir nicht versuchen, eine Softmax auf dem Vokabular von BERT zu berechnen; wir versuchen einfach, eine Softmax auf den beiden Werten zu berechnen, die BERT für die Vorhersage des nächsten Satzes zurückgibt, damit wir sehen können, welcher Wert den höchsten Wahrscheinlichkeitswert hat, und dieser wird darstellen, ob der zweite Satz eine gute Fortsetzung des ersten ist. Sobald wir die Softmax-Werte erhalten haben, können wir uns den Tensor ansehen, indem wir ihn ausdrucken.

Da der erste Wert deutlich höher ist als der zweite Index, geht das BERT davon aus, dass der zweite Satz auf den ersten Satz folgt, was die richtige Antwort ist.

# Extractive Question Answering

Bei der extraktiven Beantwortung von Fragen geht es darum, eine Frage in einem bestimmten Kontext zu beantworten, indem man die Anfangs- und Endindizes ausgibt, wo die Antwort im Kontext liegt. Hier ist mein Code für die extraktive Beantwortung von Fragen:

In [17]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
question = "What is the capital of France?"
text = "The capital of France is Paris."
inputs = tokenizer.encode_plus(question, text, return_tensors='pt')
start, end = model(**inputs)
start_max = torch.argmax(F.softmax(start, dim = -1))
end_max = torch.argmax(F.softmax(end, dim = -1)) + 1 ## add one ##because of python list indexing
answer = tokenizer.decode(inputs["input_ids"][0][start_max : end_max])
print(answer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

AttributeError: 'str' object has no attribute 'softmax'

Ähnlich wie bei den anderen drei Aufgaben laden wir zunächst das spezielle BERT-Modell für die Beantwortung von Fragen herunter und tokenisieren unsere beiden Eingaben: die Frage und den Kontext. Im Gegensatz zu den anderen Modellen ist der Prozess für dieses Modell relativ einfach, da es die Werte für jedes Wort in der tokenisierten Eingabe ausgibt. Wie bereits erwähnt, funktioniert die extraktive Beantwortung von Fragen durch die Berechnung der besten Start- und Endindizes für die Position der Antwort im Kontext. Das Modell gibt für alle Wörter im Kontext/Eingabe Werte zurück, die angeben, wie gut sie als Start- und Endwert für die gegebene Frage geeignet sind; mit anderen Worten, jedes der Wörter in der Eingabe erhält einen Start- und Endindexwert, der angibt, ob es ein gutes Startwort für die Antwort oder ein gutes Endwort für die Antwort wäre. Der Rest dieses Prozesses ist ziemlich ähnlich zu dem, was wir bei den anderen drei Programmen gemacht haben; wir berechnen die Softmax dieser Punktzahlen, um die probabilistische Verteilung der Werte zu finden, ermitteln die höchsten Werte für die Start- und End-Tensoren mit torch.argmax() und finden die tatsächlichen Token, die diesem Start- und Endbereich in der Eingabe entsprechen, und dekodieren sie und geben sie aus.

# Using BERT for any task you want

Obwohl die Textzusammenfassung, die Beantwortung von Fragen und ein grundlegendes Sprachmodell besonders wichtig sind, möchten viele Menschen BERT für andere, nicht spezifizierte Aufgaben verwenden, insbesondere in der Forschung. Dazu werden die Rohdaten der gestapelten Kodierer von BERT genommen und ein eigenes spezifisches Modell, meist eine lineare Schicht, angehängt, das dann für den jeweiligen Datensatz fein abgestimmt wird. Wenn Sie dies in Pytorch mit der Hugging Face Transformer-Bibliothek tun, ist es am besten, dies als ein Pytorch Deep Learning-Modell wie dieses einzurichten:

In [ ]:
from transformers import BertModel
class Bert_Model(nn.Module):
   def __init__(self, class):
       super(Bert_Model, self).__init__()
       self.bert = BertModel.from_pretrained('bert-base-uncased')
       self.out = nn.Linear(self.bert.config.hidden_size, classes)
   def forward(self, input):
       _, output = self.bert(**input)
       out = self.out(output)
       return out

Wie Sie sehen können, habe ich, anstatt ein spezielles BERT-Modell herunterzuladen, das bereits für eine bestimmte Aufgabe wie die Beantwortung von Fragen entwickelt wurde, das unbearbeitete, vortrainierte BertModel heruntergeladen, dem keine Köpfe zugewiesen sind.
Um die Größe der rohen BERT-Ausgaben zu erhalten, verwenden Sie einfach self.bert.config.hidden_size und fügen Sie diese der Anzahl der Klassen hinzu, die Ihre lineare Schicht ausgeben soll.

Um den obigen Code für die Sentiment-Analyse zu verwenden, was überraschenderweise eine Aufgabe ist, die in der Hugging-Face-Transformer-Bibliothek nicht heruntergeladen/bereits erledigt ist, können Sie einfach eine Sigmoid-Aktivierungsfunktion am Ende der linearen Schicht hinzufügen und die Klassen auf 1 setzen.

In [ ]:
from transformers import BertModel
class Bert_Model(nn.Module):
   def __init__(self, class):
       super(Bert_Model, self).__init__()
       self.bert = BertModel.from_pretrained('bert-base-uncased')
       self.out = nn.Linear(self.bert.config.hidden_size, classes)
       self.sigmoid = nn.Sigmoid()
   def forward(self, input, attention_mask):
       _, output = self.bert(input, attention_mask = attention_mask)
       out = self.sigmoid(self.out(output))
       return out